#Using GSTCommons to make your life easier

The GSTCommons package intended to make common tasks and definitions fast and easy.  You use GSTCommons by importing the modules you want to use.  There are three main types of modules:
1. Standard definitions,
2. Gate string list generation
3. Analysis

You typically don't import more than one of each type.

## The quick way: using a GSTCommons Analysis function
Let's first look at how to use an Analysis module which combines all the steps of running typical GST algortithms into a single function. 

In [1]:
#Import the GST module (always do this)
import GST

#Import some of the modules from the GSTCommons package (Note: don't just "import GSTCommons"):
# - import the all-in-one analysis routines that deal with gate string lists
#   that are powers of short germ strings (the most modern GST approach)
from GSTCommons import Analyze_WholeGermPowers

####First, we need to tell GST what our desired gate set is, referred to as the "target gateset".

In [2]:
#Construct a target gateset
gs_target = GST.buildGateset([2],[('Q0',)], ['Gi','Gx','Gy'], 
                             [ "I(Q0)","X(pi/2,Q0)", "Y(pi/2,Q0)"],
                             rhoExpressions=["0"], EExpressions=["1"], 
                             spamLabelDict={'plus': (0,0), 'minus': (0,-1) } )

The parameters to buildGateset, specify:
 - the state space is dimension 2 (i.e. density matrices are 2x2)
 - interpret this 2-dimensional space as that of a single qubit labeled "Q0" (label must begin with 'Q')
 - there are three gates: Idle, pi/2 x-rotation, pi/2 y-rotation
 - there is one state prep operation, which prepares the 0-state (the first basis element of the 2D state space)
 - there is one POVM (~ measurement) that projects onto the 1-state (the second basis element of the 2D state space)
 - the name of the state-prep then measure our POVM is 'plus'
 - the name of the state-prep then measure something other than our POVM is 'minus' 

To store this gateset in a file (for reference or to load it somewhere else), you just call writeGateset:

In [3]:
#Write it to a file
GST.writeGateset(gs_target, "tutorial_files/MyTargetGateset.txt")

#To load the gateset back into a python object, do:
# gs_target = GST.loadGateset("tutorial_files/MyTargetGateset.txt")

####Next, we need to create fiducial, germ, and max-length lists:
- fiducial gate strings: gate sequences that immediately follow state preparation or immediately precede measurement
- germ gate strings: gate sequences that are repeated to produce a string that is as close to some "maximum length" as possible without exceeding it.
- maxium lengths: a list of maximum lengths used to specify the increasingly long gate sequences (via more germ repeates) used by each iteration of the GST estimation procedure.

To make GST most effective, these gate strings lists must be computed.  Typically this computation is done by the Sandia GST folks and the gate string lists are sent to you.  The maximum lengths list typically starts with [0,1] and then contains successive powers of two.

In [4]:
#Create gate string lists
fiducials = GST.gateStringList( [ (), ('Gx',), ('Gy',), ('Gx','Gx'), ('Gx','Gx','Gx'), ('Gy','Gy','Gy') ])

germs = GST.gateStringList( [('Gx',), ('Gy',), ('Gi',), ('Gx', 'Gy',),
         ('Gx', 'Gy', 'Gi',), ('Gx', 'Gi', 'Gy',), ('Gx', 'Gi', 'Gi',), ('Gy', 'Gi', 'Gi',),
         ('Gx', 'Gx', 'Gi', 'Gy',), ('Gx', 'Gy', 'Gy', 'Gi',),
         ('Gx', 'Gx', 'Gy', 'Gx', 'Gy', 'Gy',)] )

#Create maximum lengths list
maxLengths = [0,1,2,4,8,16,32]

If we want to, we can save these lists in files (but not necessary):

In [5]:
GST.writeGatestringList("tutorial_files/MyFiducials.txt", fiducials, "My fiducial gate strings")
GST.writeGatestringList("tutorial_files/MyGerms.txt", germs, "My germ gate strings")

import pickle
pickle.dump( maxLengths, open("tutorial_files/MyMaxLengths.pkl", "wb"))

# To load these back into python lists, do:
#fiducials = GST.loadGatestringList("tutorial_files/MyFiducials.txt")
#germs = GST.loadGatestringList("tutorial_files/MyGerms.txt")
#maxLengths = pickle.load( open("tutorial_files/MyMaxLengths.pkl"))

#### Third, we generate (since we can't actually take) data and save a dataset
Before experimental data is obtained, it is useful to create a "template" dataset file which specifies which gate sequences are required to run GST.  Since we don't actually have an experiment for this example, we'll generate some "fake" experimental data from a set of gates that are just depolarized versions of the targets.

In [6]:
#How to create a template dataset file
lsgstLists = Analyze_WholeGermPowers.make_lsgst_lists(gs_target.keys(), fiducials, germs, maxLengths)
GST.writeEmptyDatasetFile("tutorial_files/MyDataTemplate.txt", lsgstLists[-1], "## Columns = plus count, count total")

*Note:* `lsgstLists` is a list of increasingly large and nested gate string lists, so that each list contains all the gate strings of the previous list along with (possibly) some new ones.  The last gate string list, `lsgstLists[-1]`, thus contains all the needed gate strings for GST.

Since we don't actually have a experiment to generate real data, let's now create and save a dataset using depolarized target gates and spam operations:

In [7]:
#Create a gateset of depolarized gates and SPAM relative to target, and generate fake data using this gateset.
gs_datagen = GST.GateSetTools.depolarizeGateset(gs_target, noise=0.1)
gs_datagen = GST.GateSetTools.depolarizeSPAM(gs_datagen, noise=0.001)
ds = GST.generateFakeData(gs_datagen, lsgstLists[-1], nSamples=1000000, sampleError="binomial", seed=2015)

We could at this point just use the generated dataset directly, but let's save it as though it were a file filled with experimental results.

In [8]:
#Save our dataset
GST.writeDatasetFile("tutorial_files/MyDataset.txt", lsgstLists[-1], ds)

#Note; to load the dataset back again, do:
#ds = GST.loadDataset("tutorial_files/MyDataset.txt")

#### Fourth, we call the Analysis function
Now we're all set to call the GSTCommons analysis routine.  All of the possible arguments to this function are detailed in the included help (docstring), and so here we just make a few remarks:
- For many of the arguments, you can supply either a filename or a python object (e.g. dataset, target gateset, gate string lists)
- `fiducials` is supplied twice since the state preparation fiducials (those sequences following a state prep) need not be the same as the measurement fiducials (those sequences preceding a measurement).
- Typically we want to constrain the resulting gates to be trace-preserving, so we set constrainToTP to True
- `gaugeOptRatio` specifies the ratio of the state preparation and measurement (SPAM) weight to the gate weight when performing a gauge optimization.  When this is set to 0.001, as below, the gate parameters are weighted 1000 times more relative to the SPAM parameters.  Typically it is good to weight the gates parameters more heavily since GST amplifies gate parameter errors via long gate sequences but cannot amplify SPAM parameter errors.  If unsure, 0.001 is a good value to start with.

In [9]:
results = Analyze_WholeGermPowers.doMLEAnalysis("tutorial_files/MyDataset.txt", gs_target, 
                                                 fiducials, fiducials, germs, maxLengths,
                                                 constrainToTP=True, gaugeOptRatio=1e-3)

Loading tutorial_files/MyDataset.txt: 100%
LGST: Singular values of I_tilde (truncating to first 4 of 6) = 
[  4.24408309e+00   1.16717888e+00   9.46858487e-01   9.42447698e-01
   1.23388377e-03   1.02887747e-03]

--- LGST ---

--- Gauge Optimization to TP (L-BFGS-B) ---
    3s           0.0000000000
The resulting TP penalty is: 1.36924e-13
The gauge matrix found (B^-1) is:
[[  9.99999995e-01   9.95695638e-08   5.98591798e-09  -9.15381746e-08]
 [  4.40725057e-15   1.00000000e+00  -3.31734792e-15   5.80832678e-16]
 [ -6.48950216e-15  -3.31734792e-15   1.00000000e+00   1.06274373e-14]
 [  1.92028689e-14   5.80832273e-16   1.06274369e-14   1.00000000e+00]]

The gauge-corrected gates are:
rhoVec[0] =    0.7071  -0.0214   0.0222   0.7508


EVec[0] =    0.6846   0.0022  -0.0017  -0.6436


Gi = 
   1.0000        0        0        0
  -0.0035   0.9019  -0.0004        0
   0.0033  -0.0018   0.8989  -0.0009
  -0.0032  -0.0004   0.0012   0.8995


Gx = 
   1.0000        0        0        0
  -0.00

The analysis routine returns a `GST.Results` object which encapsulates intermediate and final GST estimates, as well as quantities derived from these "raw" estimates.  (The object also caches derived quantities so that repeated queries for the same quanties do not require recalculation.)  Finally, `Results` object can generate reports and presentations containing many of the raw and derived GST results.  We give examples of these uses below. 

In [10]:
# Access to raw GST best gateset estimate
print results.gsBestEstimate

rhoVec[0] =    0.7071        0  -0.0001   0.7012


EVec[0] =    0.7071  -0.0001   0.0001  -0.7116


Gi = 
   1.0000        0        0        0
        0   0.9000   0.0002        0
        0        0   0.8999        0
        0   0.0001   0.0002   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9001   0.0001        0
        0        0        0  -0.9000
        0        0   0.9000   0.0001


Gy = 
   1.0000        0        0        0
        0        0        0   0.9000
        0        0   0.9000        0
        0  -0.9000   0.0002        0





In [11]:
#create a full GST report (most detailed and pedagogical; best for those getting familiar with GST)
results.createFullReportPDF(confidenceLevel=95, verbosity=2)

*** Generating tables ***
Generating targetSpamTable table...
Generating targetGatesTable table...
Generating datasetOverviewTable table...
Generating bestGatesetSpamTable table...
Generating bestGatesetSpamParametersTable table...
Generating bestGatesetGatesTable table...
Generating bestGatesetChoiTable table...
Generating bestGatesetDecompTable table...
Generating bestGatesetRotnAxisTable table...
Generating bestGatesetClosestUnitaryTable table...
Generating bestGatesetVsTargetTable table...
Generating bestGatesetErrorGenTable table...
Generating fiducialListTable table...
Generating rhoStrListTable table...
Generating EStrListTable table...
Generating germListTable table...
Generating logLProgressTable table...
*** Generating plots ***
 -- LogL plots (2):  1  Generating bestEstimateColorBoxPlot figure...
2  Generating invertedBestEstimateColorBoxPlot figure...

*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial ou

In [12]:
#create a brief GST report (just highlights of full report but fast to generate; best for folks familiar with GST)
results.createBriefReportPDF(confidenceLevel=95, verbosity=2)

*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial output PDF tutorial_files/MyDataset.pdf successfully generated.
Final output PDF tutorial_files/MyDataset.pdf successfully generated. Cleaning up .aux and .log files.


In [13]:
#create GST slides (tables and figures of full report in latex-generated slides; best for folks familiar with GST)
results.createPresentationPDF(confidenceLevel=95, verbosity=2)

*** Generating tables ***
*** Generating plots ***
 -- LogL plots (1):  1  
*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial output PDF tutorial_files/MyDataset.pdf successfully generated.
Final output PDF tutorial_files/MyDataset.pdf successfully generated. Cleaning up .aux and .log files.


In [14]:
#create GST slides (tables and figures of full report in Powerpoint slides; best for folks familiar with GST)
results.createPresentationPPT(confidenceLevel=95, verbosity=2)

*** Generating tables ***
*** Generating plots ***
 -- LogL plots (1):  1  
*** Assembling PPT file ***
Latexing progressTable table...
Latexing bestGatesetVsTargetTable table...
Latexing bestGatesetErrorGenTable table...
Latexing bestGatesetDecompTable table...
Latexing bestGatesetRotnAxisTable table...
Latexing bestGatesetGatesTable table...
Latexing bestGatesetSpamTable table...
Latexing bestGatesetSpamParametersTable table...
Latexing bestGatesetChoiTable table...
Latexing targetSpamTable table...
Latexing targetGatesTable table...
Latexing fiducialListTable table...
Latexing germListTable table...
Latexing datasetOverviewTable table...
Final output PPT tutorial_files/MyDataset.pptx successfully generated.


## The even quicker way: using Standard definitions with an Analysis function
A significant component of running GST as show above is defining things: the target gateset, the fiducial, germ, and maximum-length lists, etc.  We've found that many people who use GST have one of only a few different target gateset, and for these "commonly used" target gatesets we've created modules that make most of the definitions for you.  If you gateset isn't one of our "commonly used" ones then you'll have to follow the above "quick" approach for now, but please let us know and we'll try to add your gateset to the commonly used list in the future.

In the example above, our gateset (comprised of single qubit Idle, X(pi/2), Y(pi/2)) is one of the commonly used gatesets, and relevant definitions are importable via:

In [15]:
#Import the "stardard 1-qubit quantities for a gateset with X(pi/2), Y(pi/2), and idle gates"
from GSTCommons import Std1Q_XYI

We follow the same order of defining things as above, but it's much easier:

In [16]:
gs_target = Std1Q_XYI.gs_target
fiducials = Std1Q_XYI.fiducials
germs = Std1Q_XYI.germs
maxLengths = [0,1,2,4,8,16,32] #still need to define this manually

Generate a fake dataset as before:

In [17]:
gs_datagen = GST.GateSetTools.depolarizeGateset(gs_target, noise=0.1)
gs_datagen = GST.GateSetTools.depolarizeSPAM(gs_datagen, noise=0.001)
lsgstLists = Analyze_WholeGermPowers.make_lsgst_lists(gs_target.keys(), fiducials, germs, maxLengths)
ds = GST.generateFakeData(gs_datagen, lsgstLists[-1], nSamples=1000000, sampleError="binomial", seed=1234)

And run the analysis function (this time using the dataset object directly instead of loading from a file), and then create a report in the specified file.

In [18]:
results = Analyze_WholeGermPowers.doLSGSTAnalysis(ds, gs_target, fiducials, fiducials, germs, maxLengths,
                                                  constrainToTP=True)
results.createFullReportPDF(confidenceLevel=95,filename="tutorial_files/MyEvenEasierReport.pdf",verbosity=2)

LGST: Singular values of I_tilde (truncating to first 4 of 6) = 
[  4.24408716e+00   1.16704565e+00   9.46892218e-01   9.43237265e-01
   2.35094528e-03   1.17450775e-03]

--- LGST ---

--- Gauge Optimization to TP (L-BFGS-B) ---
   75s           0.0000000000
The resulting TP penalty is: 8.24295e-14
The gauge matrix found (B^-1) is:
[[  1.00000001e+00   2.28109217e-10   2.38770464e-08  -6.63069519e-08]
 [  7.25218369e-15   1.00000000e+00  -1.03702505e-16  -2.90747676e-16]
 [ -1.18208292e-15  -1.03702317e-16   1.00000000e+00  -3.58505648e-15]
 [  7.99328389e-16  -2.90747864e-16  -3.58505648e-15   1.00000000e+00]]

The gauge-corrected gates are:
rhoVec[0] =    0.7071  -0.0221   0.0217   0.7509


EVec[0] =    0.6847   0.0021  -0.0021  -0.6438


Gi = 
   1.0000        0        0        0
  -0.0033   0.9003  -0.0004  -0.0013
   0.0033  -0.0012   0.8997   0.0006
  -0.0038  -0.0004   0.0004   0.8999


Gx = 
   1.0000        0        0        0
  -0.0039   0.8996   0.0064   0.0005
   0.0003   0